### Starting a app

Now we want our application to do something rather than just creating users/groups ( **play with admin page little bit** )

First we defined our data, what we want in our application, we need these things.

- Property *what we will deal with*
  + House
  + Land
- Customer *who will these related to*
  + Seller
  + Buyer

so we will create two apps, each for

- Property
  + add/edit/delete property
  + list/show property
   
- Customer
  + either signup as seller or buyer
  + seller will be able to add new property
  + buyer will be able to view those via listing and ask for quotation/send email

*Note: since this will be simple workable demo app, this will contain basic feature of every application.*

First we will work with **property** app and finish that and go for **customer** app.

    $ ./bin/python manage.py startapp property

It will create new folder named property and some basic files on it.

In [2]:
%ll ../property

total 24
-rw-r--r-- 1 idwaker  301 Oct 12 16:06 admin.py
-rw-r--r-- 1 idwaker    0 Oct 12 14:37 __init__.py
drwxr-xr-x 3 idwaker 4096 Oct 12 15:30 migrations/
-rw-r--r-- 1 idwaker  465 Oct 12 15:50 models.py
drwxr-xr-x 2 idwaker 4096 Oct 12 16:06 __pycache__/
-rw-r--r-- 1 idwaker   60 Oct 12 14:37 tests.py
-rw-r--r-- 1 idwaker   63 Oct 12 14:37 views.py


- admin.py 

everything that we want to use with admin page goes here

- models.py

all database related code goes here

- views.py

all codes that gets data from database, works on it and creates something we can make meaning of goes here

- tests.py

all codes that tests views and models functions goes here

- migrations/

this folder contain scripts generated by django, which is used by **migrate** command to create database tables

Our House table will look like this

|Column Name|Column Type|
|-----------|-----------|
|id|Integer|
|title|Characters|
|description|text|
|price|decimal|
|address|text|
|bedrooms|Integer|
|kitchens|Integer|
|bathrooms|Integer|

we will create model using above table.

In [ ]:
# %load ../property/models.py
"""
property models
"""

from django.db import models


class House(models.Model):
    """
    creates table: house
    """
    title = models.CharField(max_length=120)
    description = models.TextField()
    price = models.DecimalField(max_digits=9, decimal_places=2)
    address = models.TextField()
    bedrooms = models.IntegerField()
    kitchens = models.IntegerField()
    bathrooms = models.IntegerField()


[Documentation for Django Model Fields](https://docs.djangoproject.com/en/1.8/ref/models/fields/)

- max_length insures that maximum length you enter for that field doesnot exceeds 120 characters
- for price maximum value that can be entered will be 9999999.99, where max_digits is total digits with decimal places 

First we need to add our **property** application to **INSTALLED_APPS** in *settings.py*
```python
INSTALLED_APPS = (
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
    'property',
)
```

Now we should create migration.

    $ ./bin/python manage.py makemigrations

this will create migrations for all installed apps, to create migration for specific app use

    $ ./bin/python manage.py makemigrations property

In [4]:
%ll ../property/migrations

total 8
-rw-r--r-- 1 idwaker  824 Oct 12 15:30 0001_initial.py
-rw-r--r-- 1 idwaker    0 Oct 12 14:37 __init__.py
drwxr-xr-x 2 idwaker 4096 Oct 12 15:35 __pycache__/


*0001_initial.py* is created by above command

In [ ]:
# %load ../property/migrations/0001_initial.py
from __future__ import unicode_literals

from django.db import migrations, models


class Migration(migrations.Migration):

    dependencies = [
    ]

    operations = [
        migrations.CreateModel(
            name='House',
            fields=[
                ('id', models.AutoField(verbose_name='ID', primary_key=True, auto_created=True, serialize=False)),
                ('title', models.CharField(max_length=120)),
                ('description', models.TextField()),
                ('price', models.DecimalField(decimal_places=2, max_digits=9)),
                ('address', models.TextField()),
                ('bedrooms', models.IntegerField()),
                ('kitchens', models.IntegerField()),
                ('bathrooms', models.IntegerField()),
            ],
        ),
    ]


we normally dont have edit/create this file manually. As you can see migration file is just a plain python class with descriptions we have added for **House** model on *property/models.py* file.

Also take note, we didnot specify **id** attribute on *property/models.py* file, which is added automatically by django model class we subclassed from.

Now, run migrate command

    $ ./bin/python manage.py migrate

tables are created, we can verify using sqlite tools such as [sqlitebrowser](http://sqlitebrowser.org/).

![database](./images/early-database.png)

still we cannot do anything with just a model, we have to enable that to admin section so that we can create/edit/delete on such models.

In [ ]:
# %load ./property/admin.py
from django.contrib import admin
from .models import House


admin.site.register(House)

we import House model and register it to admin site ( which is our admin page )

now we can see Houses added to admin homepage.
![House-Home](./images/admin-house.png)

we can add new item
![House-Add](./images/admin-house-add.png)

we can get list of items added to House model
![House-list](./images/admin-house-list.png)

let's change the House object in image above to it's title
```python
    def __str__(self):
        return self.title
```

add this method to House model (*property/models.py*) like this,
```python
class House(models.Model):
    """
    creates table: house
    """
    title = models.CharField(max_length=120)
    description = models.TextField()
    price = models.DecimalField(max_digits=9, decimal_places=2)
    address = models.TextField()
    bedrooms = models.IntegerField()
    kitchens = models.IntegerField()
    bathrooms = models.IntegerField()

    def __str__(self):
        return self.title
```

Reload the page

![House-list2](./images/admin-house-list2.png)

let's show more details, instead of just showing title on list page. edit *property/admin.py*

```python
class HouseAdmin(admin.ModelAdmin):
    list_display = ('title', 'price', 'bathrooms', 'bedrooms', 'kitchens')


admin.site.register(House, HouseAdmin)
```
![Admin-list3](./images/admin-house-list3.png)

Now let's add filter, update above class
```python
class HouseAdmin(admin.ModelAdmin):
    list_display = ('title', 'price', 'bathrooms', 'bedrooms', 'kitchens')
    list_filter = ('bathrooms', 'bedrooms', 'kitchens',)
```

![Admin-list4](./images/admin-house-list4.png)

Now again let's add search bar, we will like to search using **title**
```python
class HouseAdmin(admin.ModelAdmin):
    list_display = ('title', 'price', 'bathrooms', 'bedrooms', 'kitchens')
    list_filter = ('bathrooms', 'bedrooms', 'kitchens',)
    search_fields = ('title',)
```

![Admin-list5](./images/admin-house-list5.png)

Now let's add timestamps for our Model so that we can know when was some data created and modified.
```python
    # auto_now_add = automatically add now() [datetime.now]
    # current time when new House is added
    created_at = models.DateTimeField(auto_now_add=True)
    # auto_now = automatically add now everytime when add/updated
    updated_at = models.DateTimeField(auto_now=True)
```
we add these two attributes to House model (*property/models.py*) which looks like this finally
```python
class House(models.Model):
    """
    creates table: house
    """
    title = models.CharField(max_length=120)
    description = models.TextField()
    price = models.DecimalField(max_digits=9, decimal_places=2)
    address = models.TextField()
    bedrooms = models.IntegerField()
    kitchens = models.IntegerField()
    bathrooms = models.IntegerField()
    created_at = models.DateTimeField(auto_now_add=True)
    updated_at = models.DateTimeField(auto_now=True)

    def __str__(self):
        return self.title
```

since we have updated our **HouseModel** we need to update database as well so,

    $ ./bin/python manage.py makemigrations
    You are trying to add a non-nullable field 'created_at' to house without a default; 
    we can't do that (the database needs something to populate existing rows).
    Please select a fix:
     1) Provide a one-off default now (will be set on all existing rows)
     2) Quit, and let me add a default in models.py
    Select an option: 1

we select 1 ( which is safe to do because we don't want created_at/updated_at to be null )

    >>> timezone.now()

*Note:* Since our newly created attributes are both datetime type, we used *timezone.now()*, If it were Integers we could have used any number, for CharField any string etc.

*Note:* Remember if you are adding a ForeignKey then Integer value 1,2,3 that you use must also exists on database ie related Model should have that value as id.

*This is shown because we have already created data on database and adding new column should also have values for those rows, this oneoff value will be added automatically by migrate for those rows *

    $ ./bin/python manage.py migrate

Now let's show **created_at** on our list view

```python
class HouseAdmin(admin.ModelAdmin):
    list_display = ('title', 'price', 'bathrooms', 'bedrooms', 'kitchens',
                    'created_at',)
    list_filter = ('bathrooms', 'bedrooms', 'kitchens',)
    search_fields = ('title',)
```

our House add view now looks like this
![House-add2](./images/admin-house-list6.png)

**created_at** and **updated_at** will be updated automatically by django/database server, 